# Gradient-based planner

This notebook demonstrates how to build a gradient-based planner.

In [13]:
import os

# These environment variables control where training and eval logs are written.
# You can set these in your shell profile as well.
os.environ["RUN_DIR"] = "runs"
os.environ["EVAL_RUN_DIR"] = "eval_runs"
os.environ["MODEL_DIR"] = "models"
os.environ["DATA_DIR"] = "data"

# This is used to set a constant Tensorboard port.
os.environ["TENSORBOARD_PORT"] = str(8989)

import ml.api as ml  # Source: https://github.com/codekansas/ml-starter

# Enables logging to `stdout`.
ml.configure_logging(use_tqdm=True)

# Imports these files to add them to the model and task registry.
from usa.models.point2emb import Point2EmbModel
from usa.tasks.clip_sdf import ClipSdfTask

## Training a model

For this example, we use a clip recorded using the code in the `home-robot` repository [here](https://github.com/facebookresearch/home-robot). You can record your own clip on the Stretch robot and use that instead, by substituting the dataset path.

In [14]:
import requests
from pathlib import Path
from omegaconf import OmegaConf

data_root = Path("data")
data_root.mkdir(exist_ok=True)
dataset_path = data_root / "dataset.pkl"

# We're downloading an existing dataset, but you can use your own instead.
dataset_url = "https://github.com/codekansas/usa/releases/download/v0.0.2/chris_lab.pkl"
if not dataset_path.exists():
    with requests.get(dataset_url, stream=True) as r:
        r.raise_for_status()
        with open(dataset_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

# Using the default config, but overriding the dataset.
config = OmegaConf.load("config.yaml")
config.task.dataset = "home_robot"
config.task.dataset_path = str(dataset_path)
config.task.dataloader.train.batch_size = 2

# We're using a small number of training steps to make the example easier
# to follow, but this can be configured to improve the model quality.
config.task.finished.max_steps = 500

# We also only use the Tensorboard logger since it is easier to read.
config.logger = [{"name": "stdout"}]

# We still need to explicitly set these variables.
config.trainer.exp_name = "jupyter"
config.trainer.log_dir_name = "test"
config.trainer.base_run_dir = "runs"
config.trainer.run_id = 1

In [15]:
config.task.dataloader.train.batch_size = 2

In [16]:
objs = ml.instantiate_config(config)

# Unpacking the different components.
model = objs.model
task = objs.task
optimizer = objs.optimizer
lr_scheduler = objs.lr_scheduler
trainer = objs.trainer

#from tensorboard import notebook

# Show Tensorboard inside the notebook.
#notebook.display(port=int(os.environ['TENSORBOARD_PORT']))

# Runs the training loop.
trainer.train(model, task, optimizer, lr_scheduler)

  INFO   2023-06-02 16:30:33 [ml.trainers.base] Experiment directory: /scratch/pl2285/robot/usa/notebooks/runs/jupyter/run_1
  INFO   2023-06-02 16:30:33 [ml.core.registry] Components:
 ↪ Model: usa.models.point2emb.Point2EmbModel (/ext3/miniconda3/lib/python3.10/site-packages/usa/models/point2emb.py)
 ↪ Task: usa.tasks.clip_sdf.ClipSdfTask (/ext3/miniconda3/lib/python3.10/site-packages/usa/tasks/clip_sdf.py)
 ↪ Trainer: ml.trainers.sl.SupervisedLearningTrainer (/ext3/miniconda3/lib/python3.10/site-packages/ml/trainers/sl.py)
 ↪ Optimizer: ml.optimizers.adam.AdamOptimizer (/ext3/miniconda3/lib/python3.10/site-packages/ml/optimizers/adam.py)
 ↪ LR Scheduler: ml.lr_schedulers.linear.LinearLRScheduler (/ext3/miniconda3/lib/python3.10/site-packages/ml/lr_schedulers/linear.py)
 ↪ Launcher: ml.launchers.slurm.SlurmLauncher (/ext3/miniconda3/lib/python3.10/site-packages/ml/launchers/slurm.py)
WARNING  2023-06-02 16:30:33 [ml.trainers.base] Overwriting config /scratch/pl2285/robot/usa/notebook

## Building a planner

This example demonstrates how to use the trained model to build a planner.

In [25]:
#from usa.planners.clip_sdf import GradientPlanner
import sys
sys.path.append('../usa')
from planners.clip_sdf import GradientPlanner, AStarPlanner

# Builds the planner from the model and task. The planner
# hyperparameters can be configured as needed.
gradient_planner = GradientPlanner(
    dataset=task._dataset(),
    model=model,
    task=task,
    device=task._device,
    num_optimization_steps = 10,
    lr = 1e-2
)

grid_planner = AStarPlanner(
    dataset=task._dataset(),
    model=model,
    task=task,
    device=task._device,
    
    # The heuristic to use for AStar
    heuristic="euclidean",
    # The grid resolution
    resolution=0.1,
    # Where to store cache artifacts
    cache_dir=None,
).double()

100%|██████████| 143/143 [00:00<00:00, 356.26it/s]


In [48]:
planner = gradient_planner

In [49]:
# Generate waypoints between two explicit points.
waypoints = planner.plan(start_xy=(0, 0), end_xy=(1, 1))

# Generates waypoints from a start location to a semantic target.
#waypoints = planner.plan(start_xy=(0, 0), end_goal="The chair")

[(0, 0), (0.07773437500000036, 0.060156250000000355), (0.177734375, 0.16015625), (0.2777343750000001, 0.2601562500000001), (0.3777343750000002, 0.3601562500000002), (0.47773437500000027, 0.46015625000000027), (0.5777343750000004, 0.5601562500000004), (0.677734375, 0.66015625), (0.7777343750000001, 0.7601562500000005), (0.8777343750000002, 0.8601562500000002), (1, 1)]


100%|██████████| 10/10 [00:00<00:00, 233.71it/s]

tensor([[0.0000, 0.0000],
        [0.0778, 0.0602],
        [0.1777, 0.1602],
        [0.2778, 0.2603],
        [0.3777, 0.3601],
        [0.4778, 0.4602],
        [0.5776, 0.5601],
        [0.6777, 0.6602],
        [0.7778, 0.7603],
        [0.8779, 0.8604],
        [1.0000, 1.0000]], device='cuda:0', dtype=torch.float16,
       requires_grad=True)
tensor([[0.0000, 0.0000],
        [0.0782, 0.0622],
        [0.1771, 0.1595],
        [0.2778, 0.2603],
        [0.3777, 0.3601],
        [0.4778, 0.4602],
        [0.5776, 0.5601],
        [0.6777, 0.6602],
        [0.7773, 0.7598],
        [0.8784, 0.8623],
        [1.0000, 1.0000]], device='cuda:0', dtype=torch.float16,
       requires_grad=True)
tensor([[0.0000, 0.0000],
        [0.0790, 0.0659],
        [0.1761, 0.1587],
        [0.2778, 0.2603],
        [0.3777, 0.3601],
        [0.4778, 0.4602],
        [0.5776, 0.5601],
        [0.6777, 0.6602],
        [0.7764, 0.7588],
        [0.8799, 0.8652],
        [1.0000, 1.0000]], device='c

In [50]:
def dis(waypoints):
    total = 0
    for i in range(len(waypoints) - 1):
        total += torch.linalg.norm(torch.tensor(
            [waypoints[i + 1][0] - waypoints[i][0], waypoints[i + 1][1] - waypoints[i][1]]).float())
    return total

In [51]:
dis(waypoints)

tensor(1.4146)

In [52]:
planner.plan(start_xy=(0, 0), end_goal="The chair")

[(0, 0), (0.07773437500000036, 0.060156250000000355), (0.07773437500000036, 0.16015625), (0.177734375, 0.2601562500000001), (0.2777343750000001, 0.3601562500000002), (0.3777343750000002, 0.46015625000000027)]


100%|██████████| 10/10 [00:00<00:00, 269.74it/s]

tensor([[0.0000, 0.0000],
        [0.0778, 0.0602],
        [0.0778, 0.1602],
        [0.1777, 0.2603],
        [0.2778, 0.3601],
        [0.3777, 0.4602]], device='cuda:0', dtype=torch.float16,
       requires_grad=True)
tensor([[0.0000, 0.0000],
        [0.0699, 0.0632],
        [0.0854, 0.1586],
        [0.1771, 0.2598],
        [0.2778, 0.3601],
        [0.4202, 1.3623]], device='cuda:0', dtype=torch.float16,
       requires_grad=True)
tensor([[ 0.0000,  0.0000],
        [ 0.0569,  0.0685],
        [ 0.0980,  0.1560],
        [ 0.1643,  0.2462],
        [ 0.2869,  0.3923],
        [-1.6934,  0.8506]], device='cuda:0', dtype=torch.float16,
       requires_grad=True)
tensor([[ 0.0000,  0.0000],
        [ 0.0431,  0.0745],
        [ 0.1103,  0.1517],
        [ 0.1313,  0.2076],
        [ 0.2656,  0.4514],
        [-3.7051,  0.2231]], device='cuda:0', dtype=torch.float16,
       requires_grad=True)
tensor([[ 0.0000,  0.0000],
        [ 0.0332,  0.0800],
        [ 0.1157,  0.1439],
    

[(0.0, 0.0),
 (-0.042724609375, -0.0335693359375),
 (0.1044921875, 0.0419921875),
 (0.242431640625, -2.486328125),
 (-1.4365234375, 1.591796875),
 (-9.1875, -18.875)]

In [ ]:
from matplotlib import pyplot as plt
import torch
images = plt.figure(figsize = (50, 35))
depths = plt.figure(figsize = (50, 35))
for i, items in enumerate(task._dataset()):
    ax1 = images.add_subplot(4, 5, i + 1)
    ax1.imshow(items[0].permute(1, 2, 0))
    ax1.axis('off')
    ax2 = depths.add_subplot(4, 5, i + 1)
    #print(items[1])
    depth = items[1].clamp(1, 3)
    #ax2.imshow(items[1].permute(1, 2, 0), vmax = items[1].max(), vmin = items[1].min())
    #xi, yi = torch.meshgrid(torch.linspace(0, 1, depth.shape[1]), torch.linspace(0, 1, depth.shape[2]) )
    #ax2.contourf(xi, yi, depth[0], levels = 10000)
    ax2.imshow(depth[0])
    ax2.axis('off')
plt.colorbar()
images, depths

/ext3/miniconda3/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


WARNING  2023-06-02 16:06:06 [matplotlib.ticker] Locator attempting to generate 8702 ticks ([1.2598, ..., 3.0]), which exceeds Locator.MAXTICKS (1000).
